In [18]:
%matplotlib inline

from time import time
from math import sqrt, floor
import numpy as np
import pandas as pd
from IPython.core.debugger import set_trace

pd.options.display.float_format = '{:.3f}'.format

import matplotlib.pyplot as plt
plt.style.use = "default"

In [19]:
from IPython.core.debugger import set_trace
train = pd.read_csv("cleaned_testData1.csv")
labels = pd.read_csv("cleaned_trainLabel1.csv")

In [20]:
train = train.drop(train.columns[0], axis=1)
labels = labels.drop(labels.columns[0], axis=1)

In [21]:
def merge(df, labels):
    return labels.merge(df, left_index=True,right_index=True)

---  

# Model Building

## Which algorithm to use?
We'll use a random forest classifier (rfc) with bootstrapping and feature bagging optimizations because:
- ease of implementation
- rfcs handle multi-class predictions well without more additional effort
- works well with high dimensional data
- we'll choose use random forest as opposed to boosted trees since we have highly dimensional data
- with a reasonably high probability, can be used with the other datasets for this project since the algorithm is very robust

## The Algorithm
We'll use the CART algorithm for splitting since we have continuous data.  
  
[Full example](https://machinelearningmastery.com/classification-and-regression-trees-for-machine-learning/)  
  
Steps:
1. Initialize Tree
2. For each column, calc best split across all rows based using gini impurity score - [exmplanation](https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity) | [exmaple](https://www.researchgate.net/post/How_to_compute_impurity_using_Gini_Index) | [useful blog](http://dni-institute.in/blogs/cart-algorithm-for-decision-tree/)
3. Split the dataset based on the split condition with the highest gini score and add both sets as leaves on a tree node. The node represents a decision point, that being the condition with the highest gini score.
3. Repeat 2 & 3 until an arbitrary minimum number of rows are left
4. Prune tree

ideas:
- instead of using the raw values, categorize the numbers as # of stds away from mean
- > Alternatively, the random forest can apply weight concept for considering the impact of result from any decision tree. Tree with high error rate are given low weight value and vise versa. This would increase the decision impact of trees with low error rate - [medium post](https://medium.com/machine-learning-101/chapter-5-random-forest-classifier-56dc7425c3e1)
- [parameters to  tune](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
https://www.analyticsvidhya.com/blog/2016/04/complete-tutorial-tree-based-modeling-scratch-in-python/
- https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/
- https://stats.stackexchange.com/questions/260460/optimization-of-a-random-forest-model
- https://followthedata.wordpress.com/2012/06/02/practical-advice-for-machine-learning-bias-variance/
- https://towardsdatascience.com/beyond-accuracy-precision-and-recall-3da06bea9f6c

In [5]:
# # TEST
# def calc_best_gini_split(df, labels):
#     total_rows = df.shape[0]
    
#     def calc(col_df):
#         def g(item):
#             split1 = col_df[item > col_df]
#             split2 = col_df[item < col_df]
            
#             s1_grouped = labels.merge(pd.DataFrame(split1), left_index=True, right_index=True).groupby("label")
#             s2_grouped = labels.merge(pd.DataFrame(split2), left_index=True, right_index=True).groupby("label")
            
#             s1_group_count = s1_grouped.count()
#             s2_group_count = s2_grouped.count()
            
#             s1_cost = (1 - np.power(s1_group_count / s1_group_count.sum(), 2).sum(axis=0)) * (split1.shape[0] / total_rows)
#             s2_cost = (1 - np.power(s2_group_count / s2_group_count.sum(), 2).sum(axis=0)) * (split2.shape[0] / total_rows)

#             return (s1_cost + s2_cost).iloc[0]
        
#         # minimum cost for particular columns        
#         costs = col_df.apply(g)
#         min_cost = costs.min()
#         print(costs.shape, costs.idxmin(), costs, df)
#         min_cost_val = costs.iloc[costs.idxmin()]
        
#         return pd.Series({"cost": min_cost, "column": col_df.name, "val": min_cost_val})
    
#     splits = df.apply(calc, axis=0)
    
#     # minimum cost for whole set
#     transposed = splits.T
#     transposed["cost"] = transposed["cost"].astype(np.float32)
#     best_split = splits[transposed["cost"].idxmin()]
    
#     return best_split["column"], best_split["val"]

In [6]:
# t = time()
# calc_best_gini_split(train.iloc[:, 0:70], labels)
# time() - t

In [7]:
def calc_best_gini_split(df, labels):
    total_rows = df.shape[0]
    
    def calc(col_df):
        min_cost = 2
        min_cost_index = -1
        for index, row in col_df.iteritems():
            split1 = col_df[row > col_df]
            split2 = col_df[row < col_df]

            s1_grouped = labels.merge(pd.DataFrame(split1), left_index=True, right_index=True).groupby("label")
            s2_grouped = labels.merge(pd.DataFrame(split2), left_index=True, right_index=True).groupby("label")

            s1_group_count = s1_grouped.count()
            s2_group_count = s2_grouped.count()
            
            s1_cost = (1 - np.power(s1_group_count / s1_group_count.sum(), 2).sum(axis=0)) * (split1.shape[0] / total_rows)
            s2_cost = (1 - np.power(s2_group_count / s2_group_count.sum(), 2).sum(axis=0)) * (split2.shape[0] / total_rows)

            total_cost = (s1_cost + s2_cost).iloc[0]
            if total_cost < min_cost:
                min_cost = total_cost
                min_cost_index = index
        return pd.Series({"cost": min_cost, "index": min_cost_index})
    
    splits = df.apply(calc, axis=0)
    best_split_col = splits.T["cost"].idxmin()
    best_split_index = splits[best_split_col]["index"]
    
    return best_split_col, df[best_split_col][best_split_index]

In [8]:
# t = time()
# calc_best_gini_split(train.iloc[:, 0:70], labels)
# time() - t

In [9]:
class Node:
    def __init__(self, column, value):
        self.col = column
        self.val = value
        self.left = None
        self.right = None
    
    def traverse(self, row):
        if row[self.col] > self.val:
            return self.left
        else:
            return self.right

class DecisionTree:
    def __init__(self):
        self.root = None
    
    def grow(self, origin_df, labels, max_depth = 20, min_split_samples = 10):
        def classify_branch(df, labels):
            return labels.merge(df, left_index=True, right_index=True).groupby("label").count().sum(axis=1).idxmax()
        
        def grow_recurse(df, labels, depth):
            print("parsing @ depth {}".format(depth))
            col, val = calc_best_gini_split(df, labels)
            left_split = df[df[col] > val]
            right_split = df[df[col] < val]

            if depth > max_depth or np.minimum(left_split.shape[0], right_split.shape[0]) < min_split_samples:
                classification = classify_branch(df, labels)
                print("classified {} at depth {} to be in class {}".format(col, depth, classification))
                return classification

            node = Node(col, val)
            node.left = grow_recurse(left_split, labels, depth + 1)
            node.right = grow_recurse(right_split, labels, depth + 1)

            print("Done with depth {}".format(depth))
            return node
        
        self.root = grow_recurse(origin_df, labels, 1)
        return self
    
    def predict(self, row):
        if self.root is None: 
            raise Exception("Must call `grow` before using `predict`")
        
        node = self.root
        while type(node) is not np.int64:
            node = self.root.traverse(row)
        
        return node

In [10]:
test_row = -5
t = train.iloc[:, 0:15]

DecisionTree().grow(t, labels).predict(t.iloc[test_row]), labels.iloc[-5]

parsing @ depth 1
parsing @ depth 2
classified f4 at depth 2 to be in class 4
parsing @ depth 2
classified f7 at depth 2 to be in class 1
Done with depth 1


(1, label    1
 Name: 144, dtype: int64)

In [11]:
class RandomForest:
    def __init__(self):
        self.trees = []
    
    def grow(self, origin_df, labels, num_trees=10, num_features=None, num_sample_rows=None , max_tree_depth=20, min_split_samples=5):
        if num_features is None:
            num_features = floor(sqrt(origin_df.columns.size))
        if num_sample_rows is None:
            num_sample_rows = floor(origin_df.shape[0] / 4)

        for i in range(num_trees):
            features = np.random.choice(origin_df.columns, size=num_features, replace=False)
            rows = np.random.choice(origin_df.shape[0], size=num_sample_rows, replace=False)

            df = origin_df[features]
            df = df.iloc[rows]

            print("\n*** Creating tree #{} ***\n".format(i+1))
            self.trees.append(DecisionTree().grow(df, labels, max_tree_depth, min_split_samples))

        return self
    
    def predict(self, row):
        if not isinstance(row, pd.Series):
            raise Exception("`row` must be an instance of Pandas.Series")
            
        predictions = [tree.predict(row) for tree in self.trees]
        return np.bincount(predictions).argmax()

In [12]:
dft = train.iloc[:, 0:50]
rf = RandomForest()
rf.grow(dft, labels)


*** Creating tree #1 ***

parsing @ depth 1
parsing @ depth 2
classified f35 at depth 2 to be in class 4
parsing @ depth 2
parsing @ depth 3
classified f35 at depth 3 to be in class 1
parsing @ depth 3
parsing @ depth 4
parsing @ depth 5
classified f35 at depth 5 to be in class 1
parsing @ depth 5
classified f35 at depth 5 to be in class 1
Done with depth 4
parsing @ depth 4
classified f35 at depth 4 to be in class 1
Done with depth 3
Done with depth 2
Done with depth 1

*** Creating tree #2 ***

parsing @ depth 1
classified f23 at depth 1 to be in class 1

*** Creating tree #3 ***

parsing @ depth 1
classified f25 at depth 1 to be in class 1

*** Creating tree #4 ***

parsing @ depth 1
classified f25 at depth 1 to be in class 1

*** Creating tree #5 ***

parsing @ depth 1
classified f20 at depth 1 to be in class 1

*** Creating tree #6 ***

parsing @ depth 1
classified f45 at depth 1 to be in class 1

*** Creating tree #7 ***

parsing @ depth 1
classified f42 at depth 1 to be in clas

In [13]:
test_row = -5
rf.predict(train.iloc[test_row]), labels.iloc[test_row]

(1, label    1
 Name: 144, dtype: int64)

In [14]:
merge(dft, labels).groupby("label").describe()

f0                                                f1        ...   \
        count  mean   std   min   25%   50%   75%   max   count  mean  ...    
label                                                                  ...    
1     107.000 1.674 0.215 1.000 1.578 1.689 1.813 2.196 107.000 2.356  ...    
2      14.000 1.510 0.186 1.211 1.380 1.478 1.604 1.893  14.000 2.386  ...    
3      11.000 1.480 0.265 1.042 1.383 1.494 1.637 1.840  11.000 2.520  ...    
4      14.000 2.180 0.246 1.745 2.014 2.128 2.375 2.579  14.000 2.534  ...    
5       3.000 1.930 0.189 1.715 1.862 2.009 2.038 2.067   3.000 2.385  ...    

         f8            f9                                            
        75%   max   count  mean   std   min   25%   50%   75%   max  
label                                                                
1     1.940 2.304 107.000 2.478 0.222 1.709 2.363 2.499 2.648 2.853  
2     1.399 1.640  14.000 2.386 0.242 1.819 2.360 2.402 2.522 2.726  
3     1.949 2.109  11.000 2.401 0.412 1.366 2.411 2.511 2.618 2.811  
4     2.148 2.434  14.000 2.921 0.163 2.652 2.812 2.928 3.052 3.155  
5     2.099 2.119   3.000 2.593 0.137 2.511 2.514 2.516 2.634 2.751  

[5 rows x 400 columns]

---  
# Model Training
## Context
Now that we have our classifier, let's think about how we're going to train the model. 

We'll also measure performance through [precision](https://en.wikipedia.org/wiki/Precision_and_recall) [recall](https://towardsdatascience.com/beyond-accuracy-precision-and-recall-3da06bea9f6c) - it tells us, for each class, how well the model identifies all cases of that class (recall) and how well it can correctly classify those cases (precision). From wikipedia:
> Suppose a computer program for recognizing dogs in photographs identifies eight dogs in a picture containing 12 dogs and some cats. Of the eight dogs identified, five actually are dogs (true positives), while the rest are cats (false positives). The program's precision is 5/8 while its recall is 5/12.

![precision & recall formulas](https://cdn-images-1.medium.com/max/2000/1*6NkN_LINs2erxgVJ9rkpUA.png)
We can use the [f1 score](https://en.wikipedia.org/wiki/F1_score) to maximize precision and recall when testing different models.  
![f1 score formula](https://cdn-images-1.medium.com/max/1600/1*UJxVqLnbSj42eRhasKeLOA.png)

Recall and precision seem to be very related to bias and variance of the model, so we can maximize the f1 score by tuning the model to affect these.
#### Minimizing bias
- use new/different features
- increase the size of the trees (increases variance)
- increase the number of trees in the forest

#### Minimizing variance
- decrease the number of features
    + probably want to aim to features that are correlated and/or collapse the overall number of features through PCA
- use more data for each tree  

  
Beware: too much completixy is bad & not enough complexity is also bad  
![bias variance tradeoff](http://scott.fortmann-roe.com/docs/docs/BiasVariance/biasvariance.png)  
  

#### Stability
We need to make sure to train the classifier on as many data points as possible while also leaving enough to test to reliably tell how well the classifier actually performs. We'll use [k-fold cross validation](https://www.analyticsvidhya.com/blog/2015/11/improve-model-performance-cross-validation-in-python-r/):  
  
> 1. Randomly split your entire dataset into k ”folds”.
2. For each k folds in your dataset, build your model on k – 1 folds of the data set. Then, test the model to check the effectiveness for kth fold.
3. Record the error you see on each of the predictions.
4. Repeat this until each of the k folds has served as the test set.

## Procedure
1. Record and save an input configuration for the random forest
1. Separate data into k folds
2. For each fold *k*: 
    1. train the classifier on k-1 folds
    2. predict the k-th fold
    3. measure the: accuracy, [logarithmic](http://wiki.fast.ai/index.php/Log_Loss) [loss](https://towardsdatascience.com/metrics-to-evaluate-your-machine-learning-algorithm-f10ba6e38234#f217), recall, precision, and f1-score
3. Record the performance measures & associate it with the input configuration
3. Evaluate the overall performance difference across all configurations
4. Change **at most** 1 variable from the input configuration that optimizes perfomance & repeat steps 1-5

In [31]:
def k_folds(df, k=10):
    folds = [np.random.choice(range(df.shape[0]), 2) for i in range(k)]
    return [df.iloc[np.amin(fold): np.amax(fold)] for fold in folds]

In [33]:
t = k_folds(train)

In [36]:
[i.shape for i in t]

[(50, 687),
 (29, 687),
 (53, 687),
 (118, 687),
 (65, 687),
 (24, 687),
 (23, 687),
 (46, 687),
 (38, 687),
 (21, 687)]